In [ ]:
# | default_exp whoami.domain

In [ ]:
# | export
from meme_games.common import *

In [ ]:
# | export

@dataclass
class PlayerLabel:
    x: int = 0
    y: int = 0
    width: int = 0
    height: int = 0


@dataclass
class WhoAmIPlayer(LobbyMember):
    _ignore = LobbyMember._ignore + ('label_tfm',)
    label_text: str = ''
    label_tfm: Optional[PlayerLabel] = None
    notes: str = ''

    def set_notes(self, notes: str): self.notes = notes
    def set_label(self, label: str): self.label_text = label
    def set_label_transform(self, tfm: dict): self.label_tfm = PlayerLabel(**tfm)
    
WAILobby = Lobby[WhoAmIPlayer]

In [ ]:
p = WhoAmIPlayer(User(1,1), label_text='cat')
p.set_label_transform(dict(x=1,y=1, width=1,height=2))
asdict(p)

{'user_uid': 1,
 'lobby_id': None,
 'is_player': False,
 'is_host_': False,
 'joined_at': datetime.datetime(2025, 1, 19, 1, 14, 44, 736332),
 'score': 0,
 'id': 'scqfhpjg',
 'label_text': 'cat',
 'notes': ''}

In [ ]:
#| export
class WhoAmIManager(DataManager[WhoAmIPlayer]):
    def __init__(self, member_manager: MemberManager):
        self.mm = member_manager
        super().__init__(member_manager.db)
    
    def _set_tables(self):
        new_cols = dict(set(WhoAmIPlayer.columns().items()) - set(LobbyMember.columns().items()))
        for c in new_cols.items(): self.mm.members.add_column(*c)
        return self.mm.members
    
    def get_all(self, lobby_id: str) -> list[WhoAmIPlayer]: 
        return [WhoAmIPlayer.convert(m) for m in self.mm.get_all(lobby_id)]

In [ ]:
db = init_db()
user_manager = UserManager(db)
member_manager = MemberManager(user_manager)
lobby_manager = LobbyManager(member_manager)
whoami_manger = WhoAmIManager(member_manager)
service = LobbyService(lobby_manager)

In [ ]:
u = user_manager.create()
p = WhoAmIPlayer(u)
whoami_manger.insert(p)

WhoAmIPlayer(user=User(uid='0n8md', name='null', filename=UNSET), user_uid='0n8md', lobby_id=None, is_player=False, is_host_=False, joined_at=datetime.datetime(2025, 1, 19, 1, 14, 44, 876490), score=0, id='cyg694ey', send=None, ws=None, label_text='', label_tfm=None, notes='')

In [ ]:
lobby = service.create_lobby(user_manager.create())

In [ ]:
WhoAmIPlayer.convert(lobby.host)

WhoAmIPlayer(user=None, user_uid='rda1w', lobby_id='xiq2h', is_player=False, is_host_=True, joined_at=datetime.datetime(2025, 1, 19, 1, 14, 44, 890456), score=0, id='s3yjrffr', send=None, ws=None, label_text='', label_tfm=None, notes='')

In [ ]:
l = lobby.convert(WhoAmIPlayer)
l.host.set_label_transform()

''

In [ ]:
a = WhoAmIPlayer.convert(lobby.host)
a.set_label_transform(dict())
a

WhoAmIPlayer(user=None, user_uid='fy911', lobby_id='mwrcu', is_player=False, is_host_=True, joined_at=datetime.datetime(2025, 1, 19, 0, 21, 52, 787537), score=0, id='zqnq10o3', send=None, ws=None, label_text='', label_tfm=PlayerLabel(x=0, y=0, width=0, height=0), notes='')